In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-02-01T13:01:58.759307-08:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.21.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-1042-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [4]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 3.3 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm


from time import time

In [6]:
%watermark --iversions

pandas: 1.5.3
numpy : 1.26.3



In [7]:
ID = 'id'
TARGET = 'target'

In [8]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [9]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [10]:
y_train = train[TARGET].ravel()

train.drop([ID, TARGET, 'ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14'], axis=1, inplace=True)
test.drop([ID, 'ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14'], axis=1, inplace=True)

print(f"{train.shape},{test.shape}")
      
ntrain = train.shape[0]

(595212, 51),(892816, 51)


In [11]:
%%time
train.to_csv('../input/x_train_drop_other.csv.zip', index=False, compression='zip')
test.to_csv('../input/x_test_drop_other.csv.zip', index=False, compression='zip')

CPU times: user 19.9 s, sys: 178 ms, total: 20 s
Wall time: 20 s


In [12]:
%%time
kf = KFold(5, shuffle=True, random_state=137)

for i, (train_index, test_index) in enumerate(kf.split(train,y_train)):

        print(i)
        xgtrain, xgval = train.loc[train_index], train.loc[test_index]
        y_tr, y_val = y_train[train_index], y_train[test_index]
        
        xgtrain['target'] = y_tr
        xgval['target'] = y_val
        
        xgtrain.to_csv(f'../input/xgtrain_fold_{i}_drop_other.csv.gz', index=False, compression='gzip')
        xgval.to_csv(f'../input/xgval_fold_{i}_drop_other.csv.gz', index=False, compression='gzip')

0
1
2
3
4
CPU times: user 2min 41s, sys: 330 ms, total: 2min 42s
Wall time: 2min 42s
